# Chinese Word Segmentation
Using tokenizer, padding and binary classification

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
# import keras_tuner as kt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras. preprocessing.sequence import pad_sequences

In [ ]:
train = pd.read_csv('C:/Users/mandy/OneDrive/2022 IRS/ML/Datasets/Chinese Word Segmentation/as_training.utf8', names=['data'])
print(train.head())

## Create data and labels

In [ ]:
y = train['data'].copy()
X = train['data'].str.replace('\u3000', '')
print(X.head())
print(type(X))
print(y.head())

In [ ]:
def create_labels(data):
    label = []
    index = 0

    while index < len(data) - 1:
        if data[index + 1] == '\u3000':
            label.append(1)
            index += 2
        else:
            label.append(0)
            index += 1
            
    if index == len(data) - 1:
        label.append(1)
    return label

y = y.apply(create_labels)

In [ ]:
print(y)
print(type(y))

## Tokenization and Padding

In [ ]:
tokenizer = Tokenizer(oov_token='<OOV>', split='\u3000', char_level=True)
tokenizer.fit_on_texts(train['data'])
char_index = tokenizer.word_index
total_chars = len(json.loads(tokenizer.get_config()['word_counts']))

In [ ]:
print(list(char_index.keys())[:100])
print(len(char_index))

In [ ]:
max_length = X.str.len().max() # 188
# trunc_type = 'post'
padding = 'pre'

X_sequences = tokenizer.texts_to_sequences(X)
padded_X = pad_sequences(X_sequences, maxlen=max_length, padding=padding)
padded_y = pad_sequences(y, maxlen=max_length, padding=padding)
print(padded_X[1])
print(padded_X.shape)

In [ ]:
embedding_dim = 256

xIn = Input(shape=(max_length,))
x = Embedding(total_chars, embedding_dim, mask_zero=True)(xIn) # mask_zero will ignore timestamps with 0 (aka ignoring the padding)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)

# x = Flatten()(x) don't need to flatten, just put output layer as 2 neurons
# x = Dense(64, activation='swish')(x)
# x = Dense(64, activation='swish')(x)
x = Dense(64, activation='swish')(x)
# x = Dense(64, activation='swish')(x)
xOut = Dense(2, activation='linear')(x) # softmax is computed by loss function, so don't use activation="softmax" here

model = Model(inputs=xIn, outputs=xOut)
model.summary()

In [ ]:
class MaskedSequenceLoss(losses.Loss):
    def __init__(
        self,
        average_across_timesteps=False,
        average_across_batch=False,
        sum_over_timesteps=True,
        sum_over_batch=True,
        softmax_loss_function=None,
        name=None,
        reduction=None, # dummy arg so it can be used as custom object when loading saved model
    ):
        super().__init__()
        self.opts = {
            "average_across_timesteps": average_across_timesteps,
            "average_across_batch": average_across_batch,
            "sum_over_timesteps": sum_over_timesteps,
            "sum_over_batch": sum_over_batch,
            "softmax_loss_function": softmax_loss_function,
            "name": name,
        }
    
    def call(self, y_true, y_pred):
        return tfa.seq2seq.sequence_loss(y_pred, y_true,
                                         weights=tf.cast(y_pred._keras_mask, tf.float32) if hasattr(y_pred, "_keras_mask") else tf.ones(y_true.shape),
                                         **self.opts)

def binary_crossentropy_arg_names_changed(labels, logits):
#     print(labels.numpy(), logits.numpy())
    output = tf.nn.sigmoid_cross_entropy_with_logits(tf.cast(labels, tf.float32)[..., tf.newaxis], logits)
    print(output)
    return output

model.compile(optimizer='adam', loss=MaskedSequenceLoss(), metrics=['acc'])

In [ ]:
epochs = 100

callbacks = [
    tf.keras.callbacks.ModelCheckpoint('./8_best_model', monitor='acc', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='acc', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='acc', factor=0.1, patience=3, verbose=1)
]

padded_X = tf.convert_to_tensor(padded_X)
padded_y = tf.convert_to_tensor(padded_y)
print(type(padded_y))
history = model.fit(padded_X, padded_y, batch_size=512, epochs=epochs, validation_split=0.2, callbacks=callbacks, use_multiprocessing=True)

## Evaluate model

In [ ]:
# custom_objects = { "MaskedSequenceLoss": MaskedSequenceLoss }
# with tf.keras.utils.custom_object_scope(custom_objects):
#     model = tf.keras.models.load_model("saved-models/bidirectional-lstm/epoch8_valloss0.0042")

## Save Model

In [ ]:
import datetime as dt
dtime = dt.time()
now = dt.datetime.now()
now.strftime("%Y-%m-%d %H-%M-%S")

model.save(f'8_Chinese_Word_Segmentation/8_saved_models/{now}.h5')